<a href="https://colab.research.google.com/github/germanjke/Deep_Learning_School_MIPT/blob/master/nlp_homeworks/homework_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Spoken Language Processing
Homework 1
1. Ссылка на занятие: https://www.twitch.tv/deeplearningschool
2. Материалы: https://vk.cc/bVLCOC https://vk.cc/bVLD3Z

В этом задании предлагается обучить классификатор класса возраста по голосу (пример с тем, как это можно сделать для пола см. в семинаре)

P.S. не забудьте, что если то вы работает в Colab, то вы можете поменять среду выполнения на GPU/TPU!

Вопросы по заданию/материалам: @Nestyme

In [1]:
!pip3 install timit-utils==0.9.0

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5890 sha256=2c62570f812fed7e25127b2cec75a4948e11e57a082736ceaa2f19edee3bc00f
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [2]:
!pip3 install torchaudio

     |████████████████████████████████| 7.6MB 11.8MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101


In [3]:
! wget https://ndownloader.figshare.com/files/10256148 

--2020-12-27 11:49:37--  https://ndownloader.figshare.com/files/10256148
Resolving ndownloader.figshare.com (ndownloader.figshare.com)... 34.248.20.23, 34.255.68.79, 52.18.110.156, ...
Connecting to ndownloader.figshare.com (ndownloader.figshare.com)|34.248.20.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/10256148/TIMIT.zip [following]
--2020-12-27 11:49:38--  https://s3-eu-west-1.amazonaws.com/pfigshare-u-files/10256148/TIMIT.zip
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.36.146
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.36.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 440207227 (420M) [binary/octet-stream]
Saving to: ‘10256148’

10256148            100%[===================>] 419.81M  34.3MB/s    in 13s     

2020-12-27 11:49:51 (32.3 MB/s) - ‘10256148’ saved [440207227/440207227]



In [4]:
!unzip -q 10256148

In [5]:
import timit_utils as tu
import os
import librosa
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import Adam
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import IPython
_TIMIT_PATH = 'data/lisa/data/timit/raw/TIMIT'

## Задание 1
Загрузите данные для обучения. Для этого:
1. Скачайте датасет TIMIT (см семинар)
2. Соберите пары "голос"  — "класс возраста" также, как на семинаре собирались пары "голос"  — "пол". Аудиодорожки сконвертируйте в мелспектрограммы при помощи `torchaudio либо` `librosa`

P.S. вы можете использовать свою реализацию, а можете предложенную (см следующие ячейки)

In [6]:
import timit_utils as tu
import os
import librosa
import numpy as np
from tqdm import tqdm
import torch as t


class timit_dataloader:
    def __init__(self, data_path=_TIMIT_PATH, train_mode=True, age_mode=True):
        self.doc_file_path = os.path.join(data_path, 'DOC', 'SPKRINFO.TXT')
        self.corpus = tu.Corpus(data_path)
        with open(self.doc_file_path) as f:
            self.id_age_dict = dict(
                [(tmp.split(' ')[0], 86 - int(tmp.split('  ')[5].split('/')[-1].replace('??', '50'))) \
                 for tmp in f.readlines()[39:]])
        if train_mode:
            self.trainset = self.create_dataset('train', age_mode=age_mode)
            self.validset = self.create_dataset('valid', age_mode=age_mode)
        self.testset = self.create_dataset('test', age_mode=age_mode)

    def return_age(self, id):
        return self.id_age_dict[id]

    def return_data(self):
        return self.trainset, self.validset, self.testset

    def return_test(self):
        return self.testset

    def create_dataset(self, mode, age_mode=False):
        global people
        assert mode in ['train', 'valid', 'test']
        if mode == 'train':
            people = [self.corpus.train.person_by_index(i) for i in range(350)]
        if mode == 'valid':
            people = [self.corpus.train.person_by_index(i) for i in range(350, 400)]
        if mode == 'test':
            people = [self.corpus.test.person_by_index(i) for i in range(150)]
        spectrograms_and_targets = []
        for person in tqdm(people):
              try:
                  target = self.return_age(person.name)
                  for i in range(len(person.sentences)):
                      spectrograms_and_targets.append(
                          self.preprocess_sample(person.sentence_by_index(i).raw_audio, target, age_mode=True))
              except:
                  print(person.name, target)

        X, y = map(np.stack, zip(*spectrograms_and_targets))
        X = X.transpose([0, 2, 1])  # to [batch, time, channels]
        return X, y

    @staticmethod
    def spec_to_image(spec, eps=1e-6):
        mean = spec.mean()
        std = spec.std()
        spec_norm = (spec - mean) / (std + eps)
        spec_min, spec_max = spec_norm.min(), spec_norm.max()
        spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
        spec_scaled = spec_scaled.astype(np.uint8)
        return spec_scaled

    @staticmethod
    def clasterize_by_age(age):
        if age < 25:
            return 0
        if 25 <= age <= 40:
            return 1
        if age > 40:
            return 2

    def preprocess_sample(self, amplitudes, target, age_mode=False, sr=16000, max_length=150):
        spectrogram = librosa.feature.melspectrogram(amplitudes, sr=sr, n_mels=128, fmin=1, fmax=8192)[:, :max_length]
        spectrogram = np.pad(spectrogram, [[0, 0], [0, max(0, max_length - spectrogram.shape[1])]], mode='constant')
        target = self.clasterize_by_age(target)
        return self.spec_to_image(np.float32(spectrogram)), target

    def preprocess_sample_inference(self, amplitudes, sr=16000, max_length=150, device='cpu'):
        spectrogram = librosa.feature.melspectrogram(amplitudes, sr=sr, n_mels=128, fmin=1, fmax=8192)[:, :max_length]
        spectrogram = np.pad(spectrogram, [[0, 0], [0, max(0, max_length - spectrogram.shape[1])]], mode='constant')
        spectrogram = np.array([self.spec_to_image(np.float32(spectrogram))]).transpose([0, 2, 1])

        return t.tensor(spectrogram, dtype=t.float).to(device, non_blocking=True)


class dataloader:
    def __init__(self, spectrograms, targets):
        self.data = list(zip(spectrograms, targets))

    def next_batch(self, batch_size, device):
        indices = np.random.randint(len(self.data), size=batch_size)

        input = [self.data[i] for i in indices]

        source = [line[0] for line in input]
        target = [line[1] for line in input]

        return self.torch_batch(source, target, device)

    @staticmethod
    def torch_batch(source, target, device):
        return tuple(
            [
                t.tensor(val, dtype=t.float).to(device, non_blocking=True)
                for val in [source, target]
            ]
        )

    @staticmethod
    def padd_sequences(lines, pad_token=0):
        lengths = [len(line) for line in lines]
        max_length = max(lengths)

        return np.array(
            [
                line + [pad_token] * (max_length - lengths[i])
                for i, line in enumerate(lines)
            ]
        )

Простая сверточная сеть, ее можно дотюнить или поменять по желанию

In [7]:

import torch
import torch.nn as nn
import torch.nn.functional as F


class Model(nn.Module):
    def __init__(self, window_sizes=(3, 4, 5)):
        super(Model, self).__init__()

        self.convs = nn.ModuleList([
            nn.Conv2d(1, 128, [window_size, 128], padding=(window_size - 1, 0))
            for window_size in window_sizes
        ])

        self.fc = nn.Linear(128 * len(window_sizes), 3)

    def forward(self, x):
        # Apply a convolution + max pool layer for each window size
        x = torch.unsqueeze(x, 1)  # [B, C, T, E] Add a channel dim.
        xs = []
        for conv in self.convs:
            x2 = F.relu(conv(x))  # [B, F, T, 1]
            x2 = torch.squeeze(x2, -1)  # [B, F, T]
            x2 = F.max_pool1d(x2, x2.size(2))  # [B, F, 1]
            xs.append(x2)
        x = torch.cat(xs, 2)  # [B, F, window]

        # FC
        x = x.view(x.size(0), -1)  # [B, F * window]
        logits = self.fc(x)  # [B, class]
        #probs = torch.sigmoid(logits).view(-1)
        return logits

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'using {device} mode')
patience = 500
best_loss = 1000
cnt = 0

model = Model()
if device == torch.device('cuda'):
    model.cuda()
else:
    model.cpu()
model.train()

using cpu mode


/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


Model(
  (convs): ModuleList(
    (0): Conv2d(1, 128, kernel_size=[3, 128], stride=(1, 1), padding=(2, 0))
    (1): Conv2d(1, 128, kernel_size=[4, 128], stride=(1, 1), padding=(3, 0))
    (2): Conv2d(1, 128, kernel_size=[5, 128], stride=(1, 1), padding=(4, 0))
  )
  (fc): Linear(in_features=384, out_features=3, bias=True)
)

In [9]:
_timit_dataloader = timit_dataloader()
train, valid, test = _timit_dataloader.return_data()

trainset = dataloader(*train)
validset = dataloader(*valid)
testset = dataloader(*test)
BATCH_SIZE = 64

optimizer = Adam(
    [p for p in model.parameters() if p.requires_grad], betas=(0.9, 0.999), eps=1e-5
)

100%|██████████| 150/150 [00:17<00:00,  8.39it/s]


#Задание 2
1. Обучите свой классификатор категории возраста
2. Попробуйте улучшить результат. Можно попробовать усложнить сетку, подвигать границы категорий, поискать новые данные, что угодно, кроме учиться на тесте :)
3. Какой подход оказался самым эффективным? Как думаете, почему?
4. Как считаете, где можно было бы применить такой классификатор в качестве вспомогательной задачи?


In [36]:
criterion = nn.CrossEntropyLoss()
m = nn.Softmax(dim=1)

x = []
outs_to_f1 = np.array(x)

x = []
targets_to_f1 = np.array(x)

for i in range(5000):

    optimizer.zero_grad()
    input, target = trainset.next_batch(BATCH_SIZE, device=device)
    out = model(input)
    loss = criterion(out, target.long())
    loss.backward()
    optimizer.step()

    if i % 25 == 0:
        model.eval()

        with torch.no_grad():
            optimizer.zero_grad()

            input, target = validset.next_batch(BATCH_SIZE, device=device)
            out = model(input)
            out_ = m(out)
            out_ = torch.argmax(out_, dim=1)
            valid_loss =  criterion(out, target.long())
            out, target = out_.cpu().detach().numpy(), target.cpu().detach().numpy()

            outs_to_f1 = np.concatenate((outs_to_f1, out), axis=None)
            targets_to_f1 = np.concatenate((targets_to_f1, target), axis=None) # для проверки качества

            print("iteration {}, valid_loss {}".format(i, valid_loss.item()))

        model.train()

iteration 0, valid_loss 0.7826313376426697
iteration 25, valid_loss 1.5825917720794678
iteration 50, valid_loss 2.150235176086426
iteration 75, valid_loss 1.9359036684036255
iteration 100, valid_loss 1.2500444650650024
iteration 125, valid_loss 2.0892398357391357
iteration 150, valid_loss 1.6511121988296509
iteration 175, valid_loss 1.2036471366882324
iteration 200, valid_loss 1.9425990581512451
iteration 225, valid_loss 1.4751800298690796
iteration 250, valid_loss 1.330794334411621
iteration 275, valid_loss 1.4080251455307007
iteration 300, valid_loss 1.5300043821334839
iteration 325, valid_loss 1.2561153173446655
iteration 350, valid_loss 2.206315755844116
iteration 375, valid_loss 1.627901554107666
iteration 400, valid_loss 1.766798734664917
iteration 425, valid_loss 1.2091641426086426
iteration 450, valid_loss 2.1302788257598877
iteration 475, valid_loss 1.8829354047775269
iteration 500, valid_loss 1.8282757997512817
iteration 525, valid_loss 1.4080092906951904
iteration 550, valid

Метрикой для проверки точности классификатора была выбрана F1. Метрика считается на val датасете:

In [38]:
f1_score(outs_to_f1, targets_to_f1, average='micro')

0.6534375

А теперь метрика на test датасете

In [40]:
criterion = nn.CrossEntropyLoss()
m = nn.Softmax(dim=1)

x = []
outs_to_f1_test = np.array(x)

x = []
targets_to_f1_test = np.array(x)

for i in range(5000):
    model.eval()

    with torch.no_grad():
        optimizer.zero_grad()

        input, target = testset.next_batch(BATCH_SIZE, device=device)
        out = model(input)
        out_ = m(out)
        out_ = torch.argmax(out_, dim=1)
        valid_loss =  criterion(out, target.long())
        out, target = out_.cpu().detach().numpy(), target.cpu().detach().numpy()

        outs_to_f1_test = np.concatenate((outs_to_f1_test, out), axis=None)
        targets_to_f1_test = np.concatenate((targets_to_f1_test, target), axis=None) # для проверки качества

In [41]:
f1_score(outs_to_f1_test, targets_to_f1_test, average='micro')

0.63118125

Как увеличить точность? Самый оптимальный вариант - увеличить данные. Я, правда, не очень знаю, где найти их :( 

Попробуем поиграться с шедулерами.

Как завещал Павел Остяков в докладе "как выигрывать кагл" - шедулеры это важно. Чтож, попробуем...

Рассмотрим следующие шедулеры:


*   StepLR
*   ReduceLROnPlateau

Проверим их на валидиации, с той же метрикой F1



In [51]:
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

In [52]:
criterion = nn.CrossEntropyLoss()
m = nn.Softmax(dim=1)

x = []
outs_to_f1 = np.array(x)

x = []
targets_to_f1 = np.array(x)

for i in range(5000):

    optimizer.zero_grad()
    input, target = trainset.next_batch(BATCH_SIZE, device=device)
    out = model(input)
    loss = criterion(out, target.long())
    loss.backward()
    optimizer.step()
    scheduler.step()

    if i % 25 == 0:
        model.eval()

        with torch.no_grad():
            optimizer.zero_grad()

            input, target = validset.next_batch(BATCH_SIZE, device=device)
            out = model(input)
            out_ = m(out)
            out_ = torch.argmax(out_, dim=1)
            valid_loss =  criterion(out, target.long())
            out, target = out_.cpu().detach().numpy(), target.cpu().detach().numpy()

            outs_to_f1 = np.concatenate((outs_to_f1, out), axis=None)
            targets_to_f1 = np.concatenate((targets_to_f1, target), axis=None) # для проверки качества

            print("iteration {}, valid_loss {}".format(i, valid_loss.item()))

        model.train()

iteration 0, valid_loss 2.2966086864471436
iteration 25, valid_loss 5.340365409851074
iteration 50, valid_loss 3.0654983520507812
iteration 75, valid_loss 3.255983829498291
iteration 100, valid_loss 3.137010335922241
iteration 125, valid_loss 2.374645948410034
iteration 150, valid_loss 3.081944227218628
iteration 175, valid_loss 1.7752131223678589
iteration 200, valid_loss 2.6618635654449463
iteration 225, valid_loss 3.1778430938720703
iteration 250, valid_loss 2.621580123901367
iteration 275, valid_loss 3.8967695236206055
iteration 300, valid_loss 2.64426851272583
iteration 325, valid_loss 4.332491397857666
iteration 350, valid_loss 2.3346898555755615
iteration 375, valid_loss 2.220642328262329
iteration 400, valid_loss 2.2223293781280518
iteration 425, valid_loss 1.7765028476715088
iteration 450, valid_loss 3.6253929138183594
iteration 475, valid_loss 3.312833547592163
iteration 500, valid_loss 3.1469943523406982
iteration 525, valid_loss 3.9852426052093506
iteration 550, valid_loss 

In [53]:
f1_score(outs_to_f1, targets_to_f1, average='micro')

0.61984375

In [45]:
print('Результат ухудшился :(')

Результат ухудшился :(


Теперь ReduceLROnPlateau + ставим легендарный лёрнинг рейт Карпатого

In [49]:
optimizer = Adam(
    [p for p in model.parameters() if p.requires_grad], betas=(0.9, 0.999), eps=1e-5, lr=3e-4
)


from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, 'min')
criterion = nn.CrossEntropyLoss()
m = nn.Softmax(dim=1)

x = []
outs_to_f1 = np.array(x)

x = []
targets_to_f1 = np.array(x)

for i in range(5000):

    optimizer.zero_grad()
    input, target = trainset.next_batch(BATCH_SIZE, device=device)
    out = model(input)
    loss = criterion(out, target.long())
    loss.backward()
    scheduler.step(loss)
    optimizer.step()

    if i % 25 == 0:
        model.eval()

        with torch.no_grad():
            optimizer.zero_grad()

            input, target = validset.next_batch(BATCH_SIZE, device=device)
            out = model(input)
            out_ = m(out)
            out_ = torch.argmax(out_, dim=1)
            valid_loss =  criterion(out, target.long())
            out, target = out_.cpu().detach().numpy(), target.cpu().detach().numpy()

            outs_to_f1 = np.concatenate((outs_to_f1, out), axis=None)
            targets_to_f1 = np.concatenate((targets_to_f1, target), axis=None) # для проверки качества

            print("iteration {}, valid_loss {}".format(i, valid_loss.item()))

        model.train()

f1_score(outs_to_f1_test, targets_to_f1_test, average='micro')

iteration 0, valid_loss 2.0714356899261475
iteration 25, valid_loss 2.533285617828369
iteration 50, valid_loss 4.399380683898926
iteration 75, valid_loss 3.5106887817382812
iteration 100, valid_loss 4.764191627502441
iteration 125, valid_loss 3.0673229694366455
iteration 150, valid_loss 1.6830432415008545
iteration 175, valid_loss 2.6382737159729004
iteration 200, valid_loss 3.2412405014038086
iteration 225, valid_loss 2.2630040645599365
iteration 250, valid_loss 1.904585599899292
iteration 275, valid_loss 3.5669631958007812
iteration 300, valid_loss 2.959951639175415
iteration 325, valid_loss 2.348383665084839
iteration 350, valid_loss 3.047560691833496
iteration 375, valid_loss 2.9528329372406006
iteration 400, valid_loss 3.292499542236328
iteration 425, valid_loss 3.5191667079925537
iteration 450, valid_loss 3.7114524841308594
iteration 475, valid_loss 3.3247110843658447
iteration 500, valid_loss 4.385577201843262
iteration 525, valid_loss 3.068284273147583
iteration 550, valid_loss

0.63118125

In [50]:
f1_score(outs_to_f1, targets_to_f1, average='micro')

0.618125

In [54]:
print('Результат ухудшился :(')

Результат ухудшился :(


Попробуем lr Карпатого без шедулеров

In [55]:
optimizer = Adam(
    [p for p in model.parameters() if p.requires_grad], betas=(0.9, 0.999), eps=1e-5, lr=3e-4
)
criterion = nn.CrossEntropyLoss()
m = nn.Softmax(dim=1)

x = []
outs_to_f1 = np.array(x)

x = []
targets_to_f1 = np.array(x)

for i in range(5000):

    optimizer.zero_grad()
    input, target = trainset.next_batch(BATCH_SIZE, device=device)
    out = model(input)
    loss = criterion(out, target.long())
    loss.backward()
    optimizer.step()

    if i % 25 == 0:
        model.eval()

        with torch.no_grad():
            optimizer.zero_grad()

            input, target = validset.next_batch(BATCH_SIZE, device=device)
            out = model(input)
            out_ = m(out)
            out_ = torch.argmax(out_, dim=1)
            valid_loss =  criterion(out, target.long())
            out, target = out_.cpu().detach().numpy(), target.cpu().detach().numpy()

            outs_to_f1 = np.concatenate((outs_to_f1, out), axis=None)
            targets_to_f1 = np.concatenate((targets_to_f1, target), axis=None) # для проверки качества

            print("iteration {}, valid_loss {}".format(i, valid_loss.item()))

        model.train()

iteration 0, valid_loss 2.850710868835449
iteration 25, valid_loss 2.884730339050293
iteration 50, valid_loss 2.7052457332611084
iteration 75, valid_loss 1.9372246265411377
iteration 100, valid_loss 2.2819437980651855
iteration 125, valid_loss 1.7909961938858032
iteration 150, valid_loss 2.7792162895202637
iteration 175, valid_loss 2.276474714279175
iteration 200, valid_loss 2.5383028984069824
iteration 225, valid_loss 2.7397470474243164
iteration 250, valid_loss 4.543215274810791
iteration 275, valid_loss 2.607646942138672
iteration 300, valid_loss 2.5234498977661133
iteration 325, valid_loss 1.8086931705474854
iteration 350, valid_loss 2.4767892360687256
iteration 375, valid_loss 3.6899139881134033
iteration 400, valid_loss 2.744711399078369
iteration 425, valid_loss 2.656219720840454
iteration 450, valid_loss 3.3124282360076904
iteration 475, valid_loss 3.3163321018218994
iteration 500, valid_loss 1.364060640335083
iteration 525, valid_loss 4.3191752433776855
iteration 550, valid_lo

In [56]:
f1_score(outs_to_f1, targets_to_f1, average='micro')

0.643984375

Результат чуть ухудшился всё-таки

Также, кроме обучения на более большом датасете, есть вариант аугментации данных. Конечно, как вариант сделать больше слоёв в сетке. 

In [61]:
np.unique(outs_to_f1_test, axis=0) # проверка что предсказывает все классы

array([0., 1., 2.])

Одна из проблем - это дизбаланс класса

In [63]:
unique, counts = np.unique(outs_to_f1_test, return_counts=True)
dict(zip(unique, counts))

{0.0: 35392, 1.0: 263883, 2.0: 20725}

In [64]:
unique, counts = np.unique(targets_to_f1_test, return_counts=True)
dict(zip(unique, counts))

{0.0: 48999, 1.0: 230122, 2.0: 40879}